In [ ]:
import pandas as pd
import numpy as np
import json

# Read the Excel file
# TODO: Edit file path
file_path = "./data/GAIDs_merged_2025-03-24.xlsx"
df = pd.read_excel(file_path)

print(f"DataFrame shape: {df.shape}")
total_respondents = len(df)

In [ ]:
# 1. Country distribution (unchanged)
country_counts = df['country'].value_counts().reset_index()
country_counts.columns = ['country', 'count']
print("Country distribution:")
print(country_counts)
print(f"Total assigned to country buckets: {country_counts['count'].sum()} of {total_respondents}")


In [ ]:
# 2. Age distribution with custom buckets
# First, determine which age group each respondent belongs to
df['age_category'] = 'Prefer not to say'  # Default value

# Check if there's a single demo_age column with the age category
if 'demo_age' in df.columns and df['demo_age'].notna().any():
    # Assuming demo_age has values 1-8 corresponding to age categories
    df.loc[df['demo_age'].isin([1, 2]), 'age_category'] = '<25'  # <18 and 18-24
    df.loc[df['demo_age'] == 3, 'age_category'] = '25-34'
    df.loc[df['demo_age'] == 4, 'age_category'] = '35-44'
    df.loc[df['demo_age'] == 5, 'age_category'] = '45-54'
    df.loc[df['demo_age'].isin([6, 7, 8]), 'age_category'] = '>55'  # 55-64, 65-74, and >75
else:
    # Handle the case where age is coded in separate binary columns
    for i in range(1, 3):  # <18 and 18-24 -> <25
        col_name = f'demo_age{i}'
        if col_name in df.columns:
            df.loc[df[col_name] == 1, 'age_category'] = '<25'
    
    if 'demo_age3' in df.columns:  # 25-34
        df.loc[df['demo_age3'] == 1, 'age_category'] = '25-34'
    
    if 'demo_age4' in df.columns:  # 35-44
        df.loc[df['demo_age4'] == 1, 'age_category'] = '35-44'
    
    if 'demo_age5' in df.columns:  # 45-54
        df.loc[df['demo_age5'] == 1, 'age_category'] = '45-54'
    
    for i in range(6, 9):  # 55-64, 65-74, and >75 -> >55
        col_name = f'demo_age{i}'
        if col_name in df.columns:
            df.loc[df[col_name] == 1, 'age_category'] = '>55'
    
    # Mark explicit 'prefer not to say' responses
    if 'demo_age_prefernottosay' in df.columns:
        df.loc[df['demo_age_prefernottosay'] == 1, 'age_category'] = 'Prefer not to say'

age_counts = df['age_category'].value_counts().reset_index()
age_counts.columns = ['age_range', 'count']
age_counts = age_counts.sort_values(by='age_range', key=lambda x: pd.Categorical(
    x, categories=['<25', '25-34', '35-44', '45-54', '>55', 'Prefer not to say'], ordered=True
))
print("\nAge distribution:")
print(age_counts)
print(f"Total assigned to age buckets: {age_counts['count'].sum()} of {total_respondents}")


In [ ]:
# 3. Gender distribution
df['gender'] = 'Prefer not to say'  # Default for NA and explicit prefer not to say

# Check if there are separate binary columns for each gender
if 'demo_gender_female' in df.columns:
    df.loc[df['demo_gender_female'] == 1, 'gender'] = 'Female'
if 'demo_gender_male' in df.columns:
    df.loc[df['demo_gender_male'] == 1, 'gender'] = 'Male'
if 'demo_gender_diverse' in df.columns:
    df.loc[df['demo_gender_diverse'] == 1, 'gender'] = 'Diverse'
if 'demo_gender_prefernottosay' in df.columns:
    df.loc[df['demo_gender_prefernottosay'] == 1, 'gender'] = 'Prefer not to say'
# Check if there's a single demo_gender column with coded values
elif 'demo_gender' in df.columns and df['demo_gender'].notna().any():
    gender_mapping = {1: 'Female', 2: 'Male', 3: 'Diverse', 4: 'Prefer not to say'}
    df['gender'] = df['demo_gender'].map(gender_mapping).fillna('Prefer not to say')

gender_counts = df['gender'].value_counts().reset_index()
gender_counts.columns = ['gender', 'count']
gender_counts = gender_counts.sort_values(by='gender', key=lambda x: pd.Categorical(
    x, categories=['Female', 'Male', 'Diverse', 'Prefer not to say'], ordered=True
))
print("\nGender distribution:")
print(gender_counts)
print(f"Total assigned to gender buckets: {gender_counts['count'].sum()} of {total_respondents}")


In [ ]:
# 4. Education level distribution with custom buckets
df['education_level'] = 'Prefer not to say'  # Default for NA and explicit prefer not to say

# Check if there's a demo_bachelor column that can be used directly
if 'demo_bachelor' in df.columns and df['demo_bachelor'].notna().any():
    # Map Yes/No values to education levels
    education_mapping = {'yes': 'Bachelor and above', 'no': 'No university degree'}
    df['education_level'] = df['demo_bachelor'].map(education_mapping).fillna('Prefer not to say')
    
    # Further differentiate between Bachelor and Master+
    for i in range(9, 11):  # Bachelor level (9-10)
        col_name = f'demo_education{i}'
        if col_name in df.columns:
            df.loc[df[col_name] == 1, 'education_level'] = 'Bachelor'
    
    for i in range(11, 14):  # Master and above (11-13)
        col_name = f'demo_education{i}'
        if col_name in df.columns:
            df.loc[df[col_name] == 1, 'education_level'] = 'Master and above'
else:
    # Handle the case where education is coded in separate binary columns
    for i in range(1, 9):  # Below Bachelor
        col_name = f'demo_education{i}'
        if col_name in df.columns:
            df.loc[df[col_name] == 1, 'education_level'] = 'No university degree'
    
    for i in range(9, 11):  # Bachelor level (9-10)
        col_name = f'demo_education{i}'
        if col_name in df.columns:
            df.loc[df[col_name] == 1, 'education_level'] = 'Bachelor'
    
    for i in range(11, 14):  # Master and above (11-13)
        col_name = f'demo_education{i}'
        if col_name in df.columns:
            df.loc[df[col_name] == 1, 'education_level'] = 'Master and above'
    
    # Mark explicit 'prefer not to say' or 'other' responses
    if 'demo_education_other' in df.columns:
        # Keep as 'Prefer not to say' for simplicity, or create a separate category if needed
        pass

education_counts = df['education_level'].value_counts().reset_index()
education_counts.columns = ['education_level', 'count']
education_counts = education_counts.sort_values(by='education_level', key=lambda x: pd.Categorical(
    x, categories=['No university degree', 'Bachelor', 'Master and above', 'Prefer not to say'], ordered=True
))
print("\nEducation level distribution:")
print(education_counts)
print(f"Total assigned to education buckets: {education_counts['count'].sum()} of {total_respondents}")


In [ ]:
# Completeness check for each category
print("\nCompleteness Check:")
print(f"Country: {country_counts['count'].sum()/total_respondents:.2%}")
print(f"Age: {age_counts['count'].sum()/total_respondents:.2%}")
print(f"Gender: {gender_counts['count'].sum()/total_respondents:.2%}")
print(f"Education: {education_counts['count'].sum()/total_respondents:.2%}")

In [ ]:
# Create a new dataframe with just the 6 specified columns
df_simplified = pd.DataFrame({
    'questionnaire_id': df['questionnaire_id'],
    'AI_tech': df['AI_tech'],
    'country': df['country'],
    'age': df['age_category'],
    'gender': df['gender'],
    'education': df['education_level']
})

# Display the first few rows to check
print(f"New dataframe shape: {df_simplified.shape}")
print(df_simplified.head())

# Check for any missing values in the new dataframe
print("\nMissing values count in the simplified dataframe:")
print(df_simplified.isnull().sum())

# Make sure all records from the original dataframe are preserved
print(f"\nOriginal dataframe record count: {len(df)}")
print(f"New dataframe record count: {len(df_simplified)}")

In [ ]:
# 1. First, add the binary feeling variables (0 or 1 coded)
feeling_cols = [
    'feeling_pre_hopeful', 'feeling_pre_confident', 'feeling_pre_excited', 
    'feeling_pre_relaxed', 'feeling_pre_afraid', 'feeling_pre_angry', 
    'feeling_pre_nervous', 'feeling_pre_frustrated', 
    'feeling_pre_none of the above', 'feeling_pre_Idontknow'
]

for col in feeling_cols:
    if col in df.columns:
        # Fill NA with 0 (assuming NA means the feeling wasn't selected)
        df_simplified[col] = df[col].fillna(np.nan)
    else:
        # If the column doesn't exist, create it with all 0s
        df_simplified[col] = np.nan

# 2. Add the 1-5 scale variables with NA coded as 6 ("I don't know")
scale_cols = [
    'AIAS_life_pre', 'AIAS_work_pre', 'AIAS_futureuse_pre', 'AIAS_positive_pre',
    'AI_interest_curiosity', 'AI_interest_general', 'AI_interest_read', 'AI_interest_watchlisten'
]

for col in scale_cols:
    if col in df.columns:
        # Copy the data and ensure only values 1-5 are kept
        df_simplified[col] = df[col].copy()
        df_simplified[col] = df_simplified[col].apply(lambda x: x if x in [1, 2, 3, 4, 5] else np.nan)
    else:
        # Check if there are individual columns for each scale value (e.g., AIAS_life_pre1, AIAS_life_pre2, etc.)
        has_individual_cols = any(f"{col}{i}" in df.columns for i in range(1, 6))
        
        if has_individual_cols:
            df_simplified[col] = np.nan  # Default to NaN
            
            # Assign values based on the individual columns
            for i in range(1, 6):
                col_name = f"{col}{i}"
                if col_name in df.columns:
                    df_simplified.loc[df[col_name] == 1, col] = i
        else:
            # If neither the main column nor individual columns exist, set all to NaN
            df_simplified[col] = np.nan

# Check the first few rows to see if the new columns were added correctly
print(f"Updated dataframe shape: {df_simplified.shape}")
print(df_simplified.head())

# Check for any values outside the expected range for each type of column
print("\nChecking value ranges:")
# Binary columns should only have 0 or 1
for col in feeling_cols:
    unique_vals = sorted(df_simplified[col].unique())
    print(f"{col}: {unique_vals}")

# Scale columns should have values 1-5 (1-5 for responses, null for NA/"I don't know")
for col in scale_cols:
    unique_vals = sorted(df_simplified[col].dropna().unique(), key=lambda x: float(x) if isinstance(x, str) else x)
    print(f"{col}: {unique_vals}")

In [ ]:
# Define the groups of columns for which we'll calculate means
attitude_cols = ['AIAS_life_pre', 'AIAS_work_pre', 'AIAS_futureuse_pre', 'AIAS_positive_pre']
interest_cols = ['AI_interest_curiosity', 'AI_interest_general', 'AI_interest_read', 'AI_interest_watchlisten']

# Function to calculate mean, treating 0 ("I don't know") as a missing value
def mean_excluding_dontknow(row, columns):
    # Extract values from specified columns in this row
    values = [row[col] for col in columns]
    # Filter out NaN and "don't know" values (0)
    valid_values = [v for v in values if pd.notna(v) and v != 0]
    # Return mean if there are valid values, otherwise return NaN
    return np.nanmean(valid_values) if valid_values else np.nan

# Calculate means for each group
df_simplified['AIAS_mean_pre'] = df_simplified.apply(
    lambda row: mean_excluding_dontknow(row, attitude_cols), axis=1
)

df_simplified['AI_interest_mean'] = df_simplified.apply(
    lambda row: mean_excluding_dontknow(row, interest_cols), axis=1
)

# Round to 2 decimal places for readability
df_simplified['AIAS_mean_pre'] = df_simplified['AIAS_mean_pre'].round(2)
df_simplified['AI_interest_mean'] = df_simplified['AI_interest_mean'].round(2)

# Check the results
print("\nSummary of mean columns:")
print(f"AIAS_mean_pre: {df_simplified['AIAS_mean_pre'].describe()}")
print(f"AI_interest_mean: {df_simplified['AI_interest_mean'].describe()}")

# Check for missing values in the mean columns
print("\nMissing values in mean columns:")
print(f"AIAS_mean_pre: {df_simplified['AIAS_mean_pre'].isna().sum()}")
print(f"AI_interest_mean: {df_simplified['AI_interest_mean'].isna().sum()}")

# Sample a few rows to verify the calculation
print("\nSample verification:")
sample_indices = df_simplified.sample(3).index
for idx in sample_indices:
    row = df_simplified.loc[idx]
    print(f"\nRow {idx}:")
    
    # Print AIAS values and mean
    print("AIAS values:", end=" ")
    for col in attitude_cols:
        print(f"{col}={row[col]}", end=" ")
    print(f"→ Mean: {row['AIAS_mean_pre']}")
    
    # Print interest values and mean
    print("Interest values:", end=" ")
    for col in interest_cols:
        print(f"{col}={row[col]}", end=" ")
    print(f"→ Mean: {row['AI_interest_mean']}")

In [ ]:
# Export the simplified dataframe to JSON format
# Using 'records' orientation to create an array of objects (one per row)
df_simplified.to_json('gaid_data.json', orient='records')

# For better human readability (with indentation), you can use:
import json
with open('gaid_data_readable.json', 'w', encoding='utf-8') as f:
    json_str = df_simplified.to_json(orient='records')
    json_data = json.loads(json_str)
    json.dump(json_data, f, indent=2, ensure_ascii=False)

print("Data exported to JSON successfully!")